# dataset
Gather project IDs for vectorization task.

### Import Dependencies

In [38]:
!pip install -q --no-cache-dir mysqlclient

import os
import pandas as pd
import numpy as np
import sqlalchemy as db

### Settings & Configuration

In [39]:
DB_HOST = 'rdb0.scratch.mit.edu'
DB_PORT = 3306
DB_USER = 'haproxy_rr'
DB_PASS = 'SHlv8t1Ta1g0wjfuBbOZ'
DB_BASE = 'scratchr2_final'
DB_CONN = f'mysql+mysqldb://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BASE}'

In [73]:
# Sample size 
SAMPLE_SIZE = 600000

# Dataset paths
DATASET = './dataset'
IDS_TARGET = os.path.join(DATASET, 'project_ids') # add the .csv later.
ORIGINAL_TARGET = os.path.abspath(os.path.join(DATASET, 'dataset_unprocessed.txt'))
DATASET_TARGET = os.path.abspath(os.path.join(DATASET, 'dataset.txt'))
TRAIN_TARGET = os.path.abspath(os.path.join(DATASET, 'train')) # add the .txt later.

### Connect to Database

In [41]:
engine = db.create_engine(DB_CONN)
connection = engine.connect()

In [42]:
def query(q, **kwargs):
    q = db.text(q)
    return pd.read_sql(q, connection, coerce_float=False, params=kwargs)

----

## Get Random Sample of Project IDs
Get a random sample of *shared* project IDs.

In [52]:
def get_projects(limit):
    q = f'SELECT id FROM projects_project WHERE isPublished = TRUE ORDER BY RAND() LIMIT :limit'
    return query(q, limit=limit)

df = get_projects(SAMPLE_SIZE)

In [44]:
df = df.set_index('id')

----

### Inspect

In [53]:
len(df)

600000

In [54]:
df.sample(n=5)

,id
368385,275171007
484862,110850
273553,154241579
452574,302477864
111830,35465474


----

### split into multiple batches

In [69]:
dataframes = []
step = 60000
start = 0
end = 60000
for i in range(1,11):
    dataframe = df.iloc[start:end]
    start = end # since splicing is non-inclusive of last index
    end = (i+1)*step
    dataframe = dataframe.set_index('id')
    dataframes.append(dataframe)
    
print(len(dataframes))
display(dataframes[9])

10


""
id
1331995
272818763
321764747
64429924
88380991
...
256213757
184332448
181258045


### export batches

In [75]:
for i in range(len(dataframes)):
    dataframe = dataframes[i]
    target = IDS_TARGET + "_" + str((i+1)) + ".csv"
    dataframe.to_csv(target)

In [76]:
for i in range(len(dataframes)):
    target = IDS_TARGET + "_" + str((i+1)) + ".csv"
    !wc -l {target}

60001 ./dataset/project_ids_1.csv
60001 ./dataset/project_ids_2.csv
60001 ./dataset/project_ids_3.csv
60001 ./dataset/project_ids_4.csv
60001 ./dataset/project_ids_5.csv
60001 ./dataset/project_ids_6.csv
60001 ./dataset/project_ids_7.csv
60001 ./dataset/project_ids_8.csv
60001 ./dataset/project_ids_9.csv
60001 ./dataset/project_ids_10.csv


In [82]:
# first = 1
# second = 4
# target_1 = IDS_TARGET + "_" + str(first) + ".csv"
# target_2 = IDS_TARGET + "_" + str(second) + ".csv"
# !diff {target_1} {target_2}

----

## Extract Datasets of Different Sample Sizes

### clean and examine dataset
Clean the dataset by removing empty lines, examine text file, clean the resulting dataframe from empty / NaN rows, examine the resulting dataframe.

In [26]:
# read in dataset
with open(ORIGINAL_TARGET, "r") as f:
    lines = f.readlines()

# remove empty lines and
# write to new .txt file
with open(DATASET_TARGET, "w+") as f:
    lines = filter(str.strip, lines)
    f.writelines(lines)

In [27]:
!wc -l {DATASET_TARGET}

7889 /home/jovyan/shared/lena/dataset/dataset.txt


In [28]:
!shuf -n 5 {DATASET_TARGET}

_STARTSTACK_ event_whenkeypressed _MENU_ menu_option _MENU_ _NEXT_ pen_clear _NEXT_ pen_penDown _NEXT_ pen_setPenColorToColor _STARTINPUT_ numtext_input _ENDINPUT_ _NEXT_ procedures_call _ENDSTACK_ _STARTSTACK_ procedures_definition _NEXT_ control_repeat _STARTINPUT_ numtext_input _ENDINPUT_ _STARTNEST_ motion_movesteps _STARTINPUT_ numtext_input _ENDINPUT_ _NEXT_ motion_turnright _STARTINPUT_ numtext_input _ENDINPUT_ _NEXT_ control_wait _STARTINPUT_ numtext_input _ENDINPUT_ _ENDNEST_ _ENDSTACK_ _STARTSTACK_ event_whenkeypressed _MENU_ menu_option _MENU_ _ENDSTACK_ _STARTSTACK_ event_whenkeypressed _MENU_ menu_option _MENU_ _NEXT_ pen_clear _NEXT_ pen_penDown _NEXT_ pen_setPenColorToColor _STARTINPUT_ numtext_input _ENDINPUT_ _NEXT_ procedures_call _ENDSTACK_ _STARTSTACK_ procedures_definition _NEXT_ control_repeat _STARTINPUT_ numtext_input _ENDINPUT_ _STARTNEST_ motion_movesteps _STARTINPUT_ numtext_input _ENDINPUT_ _NEXT_ motion_turnright _STARTINPUT_ numtext_input _ENDINPUT_ _ENDNE

In [29]:
# df = pd.read_csv(DATASET_TARGET, sep="\n", header=None)
# df.columns = ['project_text']
# print(df.sample(n=5))

# read in the text file into a dataframe
def csv_to_df(filepath, columns=[]):
    df = pd.read_csv(filepath, sep="\n", header=None)
    df.columns = columns
    return df

df = csv_to_df(DATASET_TARGET, columns=['project_text'])
print(df.sample(n=5))

                                           project_text
6177  _STARTSTACK_ event_whenflagclicked _NEXT_ cont...
1910  _STARTSTACK_ event_whenflagclicked _NEXT_ look...
7193  _STARTSTACK_ event_whenflagclicked _NEXT_ even...
6444  _STARTSTACK_ event_whenflagclicked _NEXT_ soun...
2494  _STARTSTACK_ event_whenkeypressed _MENU_ menu_...


In [30]:
np.where(df.isnull())

(array([], dtype=int64), array([], dtype=int64))

### 1000 samples

In [31]:
num_samples = 1000

In [32]:
def extract_samples(dataframe, n=1000):
    # Slice the dataframe to get the top n samples
    df = dataframe.iloc[:n]
    print(df.shape)
    
    # Write the dataframe back into a text file
    target = TRAIN_TARGET + "_" + str(n) + ".txt"
    f = open(target, 'w+')
    for i in df.index:
        f.write(df.iloc[i]['project_text'] + "\n")
    
    # Return the text file handle
    return target

train_target = extract_samples(df, n=num_samples)
print(train_target)

(1000, 1)
/home/jovyan/shared/lena/dataset/train_1000.txt


In [33]:
!wc -l {train_target}

1000 /home/jovyan/shared/lena/dataset/train_1000.txt


### 10,000 samples

In [39]:
num_samples = 10000

In [40]:
train_target = extract_samples(df, n=num_samples)
print(train_target)

(10000, 1)
/home/jovyan/shared/lena/dataset/train_10000.txt


In [41]:
!wc -l {train_target}

10000 /home/jovyan/shared/lena/dataset/train_10000.txt


### 100,000 samples

In [42]:
num_samples = 100000

In [44]:
train_target = extract_samples(df, n=num_samples)
print(train_target)

(100000, 1)
/home/jovyan/shared/lena/dataset/train_100000.txt


In [45]:
!wc -l {train_target}

100000 /home/jovyan/shared/lena/dataset/train_100000.txt


### 500,000 samples

In [46]:
num_samples = 500000

In [37]:
train_target = extract_samples(df, n=num_samples)
print(train_target)

(1000, 1)
/home/jovyan/shared/lena/dataset/train_1000.txt


In [38]:
!wc -l {train_target}

1000 /home/jovyan/shared/lena/dataset/train_1000.txt
